# TASK: Forecasting demand for 5 weeks of december 2019

**Imports**

In [1]:
import pandas as pd
import numpy as np

**Data loading**

In [2]:
all_data = pd.read_csv('train.csv', delimiter=',')
all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'])
all_data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [3]:
print(f"All records: {len(all_data)}")
print(f"With demand (train): {all_data['demand'].notna().sum()}")
print(f"Without demand (test): {all_data['demand'].isna().sum()}")
print(f"\nPeriod of data: {all_data['period_start_dt'].min().date()} — {all_data['period_start_dt'].max().date()}")
print(f"\nProducts: {all_data['product_rk'].nunique()}")
print(f"Stores: {all_data['store_location_rk'].nunique()}")

All records: 35344
With demand (train): 34144
Without demand (test): 1200

Period of data: 2016-12-19 — 2019-12-30

Products: 6
Stores: 41


Analysis revealed that store 309 has non typical demand pattern - deletion increases quality of a model

In [4]:
store_309 = all_data[all_data['store_location_rk'] == 309]
print(f"Store 309:")
print(f"   Records: {len(store_309)}")
print(f"   Missings in demand: {store_309['demand'].isna().sum()}")
print(f"   First date: {store_309['period_start_dt'].min().date()}")

Store 309:
   Records: 15
   Missings in demand: 0
   First date: 2016-12-19


In [5]:
all_data = all_data[all_data['store_location_rk'] != 309].copy()

In [6]:
print(f"\nStore 309 deleted. Remaining records: {len(all_data)}")


Store 309 deleted. Remaining records: 35329


**Data split**

In [7]:
train_data = all_data[all_data['demand'].notna()].copy()
test_data = all_data[all_data['demand'].isna()].copy()

In [8]:
print(f" Train: {len(train_data)} records")
print(f" Test: {len(test_data)} records")

print(f"\n Test weeks (for prediction):")
for dt in sorted(test_data['period_start_dt'].unique()):
    week_num = (dt.day - 1) // 7 + 1
    print(f"   {dt.date()} - week {week_num} of december")

 Train: 34129 records
 Test: 1200 records

 Test weeks (for prediction):
   2019-12-02 - week 1 of december
   2019-12-09 - week 2 of december
   2019-12-16 - week 3 of december
   2019-12-23 - week 4 of december
   2019-12-30 - week 5 of december


# Helper functions

Returns number of week in december

In [9]:
def get_dec_week(dt):
    return (dt.day - 1) // 7 + 1

Finds combinations (product, store) with 0 demand over last n weeks

It may mean that product was discontinued in the store, is out of stock and will not appear again, or there is no demand for product in the store

In such cases forecast = 0 is better than a model prediction

- experiments showed that n_weeks=5 is the best

In [10]:
def find_zero_combinations(train_df, n_weeks):
    train_df = train_df.sort_values(['product_rk', 'store_location_rk', 'period_start_dt'])
    last_date = train_df['period_start_dt'].max()
    cutoff = last_date - pd.Timedelta(weeks=n_weeks-1)
    recent = train_df[train_df['period_start_dt'] >= cutoff]
    zero_combos = recent.groupby(['product_rk', 'store_location_rk'])['demand'].sum()
    return set(zero_combos[zero_combos == 0].index.tolist())

# Patterns

**Pattern preparation for december**

December 2018 - best base

- It is the last full december before foorecast period
- Unique seasonality (normal demand -> growth -> peak demand due to New Year)

December 2018 is used for baseline demand level for product x store pairs and weekly patterns within december

In [11]:
dec_2018 = train_data[(train_data['period_start_dt'] >= '2018-12-01') &
                       (train_data['period_start_dt'] <= '2018-12-31')].copy()
dec_2018['dec_week'] = dec_2018['period_start_dt'].apply(get_dec_week)

In [12]:
print(f" December 2018: {len(dec_2018)} records")
print(f"\n Mean demand by december 2018 weeks:")

week_demand = dec_2018.groupby('dec_week')['demand'].mean()
for week, demand in week_demand.items():
    print(f"   Week {week}: {demand:.2f}")

 December 2018: 1185 records

 Mean demand by december 2018 weeks:
   Week 1: 9.04
   Week 2: 11.42
   Week 3: 19.58
   Week 4: 37.00
   Week 5: 39.24


**Global and product patterns**

*Global* - average ratio of each week to total month (for all products)
- Used as a fallback if there is no product specific dataavailable

*Product level* — ratio for each product calculated separately
- Different products show different patterns in december

In [13]:
global_mean = dec_2018['demand'].mean()
print(f"Global mean december 2018: {global_mean:.2f}")

Global mean december 2018: 23.37


In [14]:
global_week_ratios = (dec_2018.groupby('dec_week')['demand'].mean() / global_mean).to_dict()

print(f"\n Global week coefficients:")
for week, ratio in global_week_ratios.items():
    trend = "up" if ratio > 1 else "down"
    print(f"   Week {week}: {ratio:.3f} {trend}")


 Global week coefficients:
   Week 1: 0.387 down
   Week 2: 0.489 down
   Week 3: 0.838 down
   Week 4: 1.584 up
   Week 5: 1.679 up


In [15]:
product_dec_mean = dec_2018.groupby('product_rk')['demand'].mean().to_dict()

product_week_ratios = {}

# coefficient = demand in this week / mean demand for a product in a month
for (prod, week), demand in dec_2018.groupby(['product_rk', 'dec_week'])['demand'].mean().items():
    product_week_ratios[(prod, week)] = demand / product_dec_mean[prod]

**YOY trend - year over year**

Comparing november 2019 with november 2018 in order to know
- Demand has increased or decreased for each product x store pair
- By what percentage

November is used as it is last month before forecast (it is known)

TREND_WEIGHT = 0.30

In [16]:
nov_2019 = train_data[(train_data['period_start_dt'] >= '2019-11-01') &
                       (train_data['period_start_dt'] <= '2019-11-30')]
nov_2018 = train_data[(train_data['period_start_dt'] >= '2018-11-01') &
                       (train_data['period_start_dt'] <= '2018-11-30')]

# mean demand for each combination product x store
nov_2019_agg = nov_2019.groupby(['product_rk', 'store_location_rk'])['demand'].mean()
nov_2018_agg = nov_2018.groupby(['product_rk', 'store_location_rk'])['demand'].mean()

In [17]:
# trend = november 2019 / november 2019
# bound values from 0.3 to 3.0 - to avoid extreme values
yoy_trend = (nov_2019_agg / nov_2018_agg.replace(0, np.nan)).fillna(1.0).clip(0.3, 3.0).to_dict()

In [18]:
trend_values = list(yoy_trend.values())
print(f"\n Statistics of YoY trend:")
print(f"   Mean: {np.mean(trend_values):.3f}")
print(f"   Median: {np.median(trend_values):.3f}")
print(f"   Min: {np.min(trend_values):.3f}")
print(f"   Max: {np.max(trend_values):.3f}")


 Statistics of YoY trend:
   Mean: 0.938
   Median: 0.929
   Min: 0.300
   Max: 3.000


In [19]:
if np.mean(trend_values) < 1:
    print(f"\n Mean trend < 1 -> demand in 2019 lower than in 2018")


 Mean trend < 1 -> demand in 2019 lower than in 2018


**Preparation of base demand**

For each combination product x store determine the baseline demand level

Priority of sources
-  Average from December 2018 - best source
- Average from the latest data × December/November coeff - fallback
- Product level average - if no store specific data available
- Global average

In [20]:
base_demand = dec_2018.groupby(['product_rk', 'store_location_rk'])['demand'].mean().to_dict()
print(f"Base from december 2018: {len(base_demand)} combinations")

Base from december 2018: 239 combinations


In [21]:
recent = train_data[train_data['period_start_dt'] >= '2019-10-01']
recent_base = recent.groupby(['product_rk', 'store_location_rk'])['demand'].mean().to_dict()
print(f"Fallback from recent data: {len(recent_base)} combinations")

Fallback from recent data: 240 combinations


In [22]:
dec_to_nov_ratio = dec_2018['demand'].mean() / nov_2018['demand'].mean()
print(f"Coefficient december/november: {dec_to_nov_ratio:.3f}")

Coefficient december/november: 2.175


# Parameters

Rule for zeros - if the product was not sold for n weeks in the store it is likely that it would not be sold in december too

In [23]:
ZERO_WEEKS = 5

zero_combinations = find_zero_combinations(train_data, n_weeks=ZERO_WEEKS)

In [24]:
print(f"Combinations with 0 demand: {len(zero_combinations)}")
print(f"{len(zero_combinations) / len(test_data) * 100:.1f}% from test records")

Combinations with 0 demand: 35
2.9% from test records


**Week corrections**

These coefficients adjust the forecast for each week of december

In [25]:
WEEK_CORRECTIONS = {1: 0.92, 2: 0.92, 3: 0.89, 4: 0.97, 5: 1.06}

Weight for YOY trend

In [26]:
TREND_WEIGHT = 0.30

# Forecast

In [27]:
predictions = []

for _, row in test_data.iterrows():
    product = row['product_rk']
    store = row['store_location_rk']
    week = get_dec_week(row['period_start_dt'])
    combo = (product, store)

    # zero rule
    if combo in zero_combinations:
        predictions.append(0)
        continue

    # base
    base = base_demand.get(combo)
    if base is None:
        base = recent_base.get(combo)
        if base is not None:
            base = base * dec_to_nov_ratio
        else:
            base = product_dec_mean.get(product, global_mean)

    # week pattern
    week_ratio = product_week_ratios.get((product, week), global_week_ratios.get(week, 1.0))

    # trend
    trend = yoy_trend.get(combo, 1.0)
    trend_adjusted = 1 + (trend - 1) * TREND_WEIGHT

    # final prediction
    pred = base * week_ratio * WEEK_CORRECTIONS[week] * trend_adjusted
    predictions.append(max(pred, 0))

Save submission

In [28]:
submission = test_data[['id']].copy()
submission['predicted'] = predictions
submission.to_csv('submission.csv', index=False)

# Path to the final solution

**Step 1 - initial approach — ML with lag features**

*What I tried*

LightGBM with 606 features

- lags 1–4 weeks and 1 year
- rolling means (14D, 28D, 56D, 84D)
- EWMs with different alphas
- aggregations

Validation was on November 2019

*Results*

SMAPE around 79 — very poor

*Why it didn’t work*

- Overfitting - 606 features on 34k records — model captured noise
- Lag features became NaN for forecast weeks — model loose information
- December has unique seasonality which lags cannot capture

**Step 2 - model simplification**

*What I tried*
- Reducing feature set from 606 to around 50 - only basic lags and calendar f eatures

*Results*

SMAPE 68 — improvement

*Why it worked*

Fewer features -> less noise -> less overfitting.

**Step 3 - december seasonality analysis**

*From analysis*
- December - significantly different from other months — demand rises toward the  New Year
- Last full december in dataset - 2018.
- Pattern within december: week 1 (9 units), week 5 (39 units) — approximately 4x increase

*What I tried*
- I used december 2018 as a base for forecast
- Calculated week_ratio = weekly_demand / monthly_demand
- Formula: predicted = base x week_ratio

*Results*
- SMAPE 62 — another improvement

*Conclusion*
December 2018 - best predictor of december 2019 as
- closest full december
- contains the same New Year seasonality
- has the same stores and products

**Step 4 - add YoY trend**

*From analysis*
- demand in 2019 may change compared to 2018 (store growth/decline)
- november 2019 compared to November 2018 reflects current state of each pair product x store

*What I tried*
- calculated yoy_trend = nov_2019 / nov_2018 - for each combination
- capped it to [0.3, 3.0] - to avoid extremes
- predicted = base x week_ratio x trend

*Results*
- with full trend (100%) - performance worsened — the trend is too noisy
- with trend_weight = 0.3: SMAPE 57 — an improvement

**Step 5 - grid search for weekly corrections

*From analysis*
- model overestimate the beginning of the month and underestimate the end
- weekly correction coefficients are needed

*What I tried*
- grid search over w1, w2, w3, w4, w5 in the range [0.90, 1.10]
- combinations - evaluated using average predictions and validation

*Results*
- Best combination: {1: 0.96, 2: 0.96, 3: 0.94, 4: 0.98, 5: 1.02}
- SMAPE 56 — small improvement

*Why it worjed*
- week 3 requires strong reduction — base model seems to overestimate it
- week 5 needs an increase — the New Year peak was underestimated

**Step 6 - zero rule — main breakthrough**

*Logic*

If the product was not sold for n weeks in a store then most likely

- product has been discontinued
- no restocks
- no demand in this store

In all cases forecast=0 - more accurate than models

*What I tried*
- implemented rule with n_weeks = 8 initially
- found 165 combinations with 0 demand - forecast to 0

*Results*
- SMAPE 50 —  beat 1 benchmark

*Why it worked*
- SMAPE heavily penalizes forecasts > 0 if the real value is 0
- setting false positives to zero - removes penalties

**Step 6.1 - zero rule optimisation**

*What I tried*
- n_weeks=2, 3, 4, 5, 6, 7, 8, 10, 12

*Results*
- 5 is the best for private score, 2 - best for public

**Step 8 - optimisation of other parameters**

*What I tried*
- grid search: w1, w2, w3, w4, w5, trend_weight with fixed zero_weeks=5

*Final parameters*

- WEEK_CORRECTIONS = {1: 0.92, 2: 0.92, 3: 0.89, 4: 0.97, 5: 1.06}
- TREND_WEIGHT = 0.30
- ZERO_WEEKS = 5

*Result*

- Private SMAPE=48.024 — best result

# What did not work

1. CatBoost and recursive forecasting  

Logic: predict week by week with previous forecasts as lag features

Result: SMAPE 74–76 — bad

Why: Accumulation of errors (week 1 error -> distorted lags -> 2 error -> etc)

2. Promo correction  

Logic: PROMO1_FLAG in test - hint. Increase forecast during promo

Result: SMAPE 81 — very bad

Why: promo multiplier was x3 — too aggressive - it broke the model

3. Product or store level corrections  

Logic: different coeffs - for different products/stores

Result: 57 — not better than base model

Why: not enough data to estimate individual coeffs

4. Quantile regression  

Logic: median instead of mean - better for SMAPE

Result: 57 — no improvement

Why: in this dataset median ≈ mean.

5. Ensembles of ML models

Idea: average several CatBoost models with different seeds

Result: 74–76 — still bad

Why: If base approach is bad - ensemble will not save it

6. weighting multiple Decembers (2016, 2017, 2018)  

Idea: average patterns across all dcembers

Result: 79 — worse than using just 2018

Why: older data is less relevant, 2018 is closest to 2019

# Key findings

- simplicity - 6 parameters instead of 606 features

- domain knowledge is critical - zero rule is business logic, not math

- december is unique - we cannot make predictions from other months

- smoothing - 30% of the trend works better than 100

- grid search - works on simple models

- ML can work badly on small data with strong seasonality

# Final solution

**zero rule**

if sum of demand for 5 weeks == 0 -> predicted = 0

else predicted = base_dec2018 x week_ratio x week_correction x trend_adjusted

**parameters**

WEEK_CORRECTIONS = {1: 0.92, 2: 0.92, 3: 0.89, 4: 0.97, 5: 1.06}

TREND_WEIGHT = 0.30

ZERO_WEEKS = 5